Questions - 
1. what is year? - 2024 Curriculum or just 2024?
2. Introduction Summary? - is it entire summary section or do we need to use some library to summarize the para
3. Learning 
Outcome? - do we need it with the format, or just the text
4. Link to the Summary Page? - whis is this could not find it on the sites

In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
import logging
from IPython.display import display, Markdown
import pandas as pd
import requests
from selenium import webdriver
from typing import List, Tuple

In [2]:
# Configure logging
file_name = ".\logs\webscrapping.log"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
if not logger.handlers:
    handler = logging.FileHandler(file_name)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# display log message in-line 
class NotebookHandler(logging.Handler):
    def emit(self, record):
        display(Markdown(self.format(record)))
        
logger.addHandler(NotebookHandler())

# set the file to empty at start
with open(file_name, 'w'):
    pass

In [3]:
# test the URL connection
try:
    logger.info('Testing URL connection for Web Scrapping')
    url_test = "https://www.cfainstitute.org/en/membership/professional-development/refresher-readings#first={first}&sort=%40refreadingcurriculumyear%20descending&numberOfResults=100".format(first=0)
    print("URL: ",url_test)
    response_test = requests.get(url_test)
    print(response_test)
    logger.info('Successfully connected to the site')
except:
    logger.error("Error in loading the URL")

Testing URL connection for Web Scrapping

URL:  https://www.cfainstitute.org/en/membership/professional-development/refresher-readings#first=0&sort=%40refreadingcurriculumyear%20descending&numberOfResults=100
<Response [200]>


Successfully connected to the site

In [10]:
# function to return individual page Links
def getPageURLs() -> list:
    # base URL
    base_url = "https://www.cfainstitute.org/en/membership/professional-development/refresher-readings"
    
    # additional section for the URL to specify the first row and the number of results
    additional_section = "#first={first}&sort=%40refreadingcurriculumyear%20descending&numberOfResults=100"
    
    individual_topics_urls = [] # store individual page URLs (sub pages to collect data)
    first_entry = 0 # start of the entries on web page
    has_more_pages = True # flag to keep scrapping
    count = 0
    logger.info("------- Starting URL Extraction -------")
    while has_more_pages:
        try:
            # increment counter
            count += 1
            
            # Set up Selenium WebDriver to scrape dynamic data
            driver = webdriver.Chrome()
        
            # Construct URL to Scrape Data
            url = base_url + additional_section.format(first=first_entry)  
        
            # load the Web Page that has Dyanmic content with help of selenium 
            driver.get(url)
        
            # get the source HTML code
            page = driver.page_source
        
            # use BeautifulSoup to load pase the content
            parsed_content = bs(page, 'html.parser')
        
            # find all the individual 224 entries and get the URL
            for parse in parsed_content.find_all("div", {"class": "coveo-list-layout"}):
                a_tag = parse.find('a') # get the <a> tags
                individual_topics_urls.append((a_tag.text, a_tag.get('href')))
    
            # Find the Next page button on the page
            next_button = parsed_content.find_all("li", {"class": "coveo-pager-next"})
    
            # if next button is present go to next page 
            if next_button:
                first_entry += 100
            else:
                has_more_pages = False
                
            # Close the Selenium WebDriver
            driver.quit()
            
            logger.info("Total topics found after iteration {count} iterations are {length}".format(count=count, length=len(individual_topics_urls)))
            
        except Exception as e:
            logger.error("Exception:", e)
            break

    logger.info("------- Ending URL Extraction -------")
    return individual_topics_urls    

In [11]:
li = getPageURLs()

------- Starting URL Extraction -------

Total topics found after iteration 1 iterations are 100

Total topics found after iteration 2 iterations are 200

Total topics found after iteration 3 iterations are 224

------- Ending URL Extraction -------

In [172]:
# function to scrape Data from CFA site (Raw Data)
def extractData(individual_topics_urls: List[Tuple[str, str]]):
    
    # Set up Selenium WebDriver to scrape dynamic data
    driver = webdriver.Chrome()
    
    # initialize data frame
    raw_df = pd.DataFrame(columns=['NameOfTheTopic', 'Year', 'Level', 'IntroductionSummary', 'LearningOutcomes', 'SummaryPageLink', 'PDFFileLink'])

    # iteration counter
    counter = 0
    
    for topic, topic_url in individual_topics_urls[:10]:
        try:
            # load the Web Page that has Dyanmic content with help of selenium 
            driver.get(topic_url)

            # reset temp
            temp = {}

            # increment counter
            counter += 1

            logger.debug("Iteration {} topic {} --->".format(counter, topic))
            
            # get the source HTML code
            page = driver.page_source

            # use BeautifulSoup to load pase the content
            parsed_content = bs(page, 'html.parser')

            # extract title
            title = parsed_content.find("h1", {"class": "article-title"}).text.strip()

            # extract Year and Level
            content_utility = parsed_content.find("div", {"class": "content-utility"})
            year = content_utility.find('span', {"class": "content-utility-curriculum"}).text.strip().split('\n')[0]
            level = content_utility.find("span", {"class": "content-utility-topic"}).text.strip()

            # extract Introduction Summary and Learning Outcomes
            h2_elements = parsed_content.find_all("h2", {"class": "article-section"})
            
            # h2_elements[0] ---> for introduction
    
            learning_outcomes = h2_elements[1].find_next_sibling().find_all("li") 
            learnings = ''
            for learning_outcome in learning_outcomes:
                if learning_outcome.text:
                    learnings += learning_outcome.text
                else: 
                    learnings += learning_outcome.find('p').text.strip()
            
            learning_list = [s.strip() for s in learnings.split("\n")]
            learnings = ''.join(learning_list)
            
            # extrat PDF File Link
            lock_content = parsed_content.find("section", {"class": "primary-asset login-required"})
            pdf_link = [a for a in lock_content.find_all('a', {"class": "locked-content"}) if a.text.strip()=='Download the full reading (PDF)'][0].get("href")  
            pdf_link = 'https://www.cfainstitute.org' + pdf_link
            temp['NameOfTheTopic'] = title
            temp['Year'] = year
            temp['Level'] = level
            temp['LearningOutcomes'] = learnings
            temp['PDFFileLink'] = pdf_link

            raw_df = pd.concat([raw_df, pd.DataFrame([temp])], ignore_index=True)

        
        except Exception as e:
            logger.error("Exception:", e)
            
    # Close the Selenium WebDriver
    driver.quit()

    return raw_df
    

In [173]:
df = extractData(li)

Iteration 1 topic Time-Series Analysis  --->

Iteration 2 topic Credit Analysis Models  --->

Iteration 3 topic Introduction to Alternative Investments  --->

Iteration 4 topic Credit Default Swaps  --->

Iteration 5 topic Valuation of Contingent Claims  --->

Iteration 6 topic Introduction to Commodities and Commodity Derivatives  --->

Iteration 7 topic Understanding Income Statements  --->

Iteration 8 topic Pricing and Valuation of Forward Commitments  --->

Iteration 9 topic Private Equity Investments  --->

Iteration 10 topic Valuation and Analysis of Bonds with Embedded Options  --->

In [174]:
df

,NameOfTheTopic,Year,Level,IntroductionSummary,LearningOutcomes,SummaryPageLink,PDFFileLink
0,Time-Series Analysis,2024,Level II,NaN,calculate and evaluate the predicted trend val...,NaN,https://www.cfainstitute.org/-/media/documents...
1,Credit Analysis Models,2024,Level II,NaN,"explain expected exposure, the loss given defa...",NaN,https://www.cfainstitute.org/-/media/documents...
2,Introduction to Alternative Investments,2023,Level I,NaN,describe types and categories of alternative i...,NaN,https://www.cfainstitute.org/-/media/documents...
3,Credit Default Swaps,2024,Level II,NaN,"describe credit default swaps (CDS), single-na...",NaN,https://www.cfainstitute.org/-/media/documents...
4,Valuation of Contingent Claims,2024,Level II,NaN,describe and interpret the binomial option val...,NaN,https://www.cfainstitute.org/-/media/documents...
5,Introduction to Commodities and Commodity Deri...,2024,Level II,NaN,compare characteristics of commodity sectors;c...,NaN,https://www.cfainstitute.org/-/media/documents...
6,Understanding Income Statements,2023,Level I,NaN,describe the components of the income statemen...,NaN,https://www.cfainstitute.org/-/media/documents...
7,Pricing and Valuation of Forward Commitments,2024,Level II,NaN,describe the carry arbitrage model without und...,NaN,https://www.cfainstitute.org/-/media/documents...
8,Private Equity Investments,2023,Level II,NaN,explain sources of value creation in private e...,NaN,https://www.cfainstitute.org/-/media/documents...
9,Valuation and Analysis of Bonds with Embedded ...,2024,Level II,NaN,describe fixed-income securities with embedded...,NaN,https://www.cfainstitute.org/-/media/documents...
